## General review
documentation impressions
- https://opensearch.org/docs/latest/getting-started/intro/ 
- focused on document search?
- "OpenSearch uses the BM25 ranking algorithm to calculate document relevance scores and then returns the results sorted by relevance."

They do care about document search, relevance things, etc.

UI
- open http://localhost:5601/


- https://github.com/opensearch-project/opensearch-py/blob/main/USER_GUIDE.md#creating-an-index


## Hybrid search review
- https://opensearch.org/docs/latest/search-plugins/hybrid-search/#step-1-create-an-ingest-pipeline

- Insert data -> not good (should config pipelines or something)
- Documentation good
- Configure model -> good <hr>


request
```
GET /my-nlp-index/_search?search_pipeline=nlp-search-pipeline
{
  "_source": {
    "exclude": [
      "passage_embedding"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "text": {
              "query": "cowboy rodeo bronco"
            }
          }
        },
        {
          "neural": {
            "passage_embedding": {
              "query_text": "wild west",
              "model_id": "CM6SGZIBXZwX3jIm7TxM",
              "k": 5
            }
          }
        }
      ]
    }
  }
}
```

response
```
{
  "took": 366,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 3,
      "relation": "eq"
    },
    "max_score": 0.3,
    "hits": [
      {
        "_index": "my-nlp-index",
        "_id": "3",
        "_score": 0.3,
        "_source": {
          "text": "People line the stands which advertise Freemont 's orthopedics , a cowboy rides a light brown bucking bronco .",
          "id": "2664027527.jpg"
        }
      },
      {
        "_index": "my-nlp-index",
        "_id": "5",
        "_score": 0.26955464,
        "_source": {
          "text": "A rodeo cowboy , wearing a cowboy hat , is being thrown off of a wild white horse .",
          "id": "2691147709.jpg"
        }
      },
      {
        "_index": "my-nlp-index",
        "_id": "4",
        "_score": 0.0003,
        "_source": {
          "text": "A man who is riding a wild horse in the rodeo is very near to falling off .",
          "id": "4427058951.jpg"
        }
      }
    ]
  }
}
```

In [1]:
from opensearchpy import OpenSearch

In [19]:
host = 'localhost'
port = 9200

# Create the client with SSL/TLS and hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth=('admin', 'Kwonjiwon411!'),
    http_compress = True, # enables gzip compression for request bodies
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

# upload mapping
curl -H "Content-Type: application/x-ndjson" -X PUT "https://localhost:9200/test" -ku admin:<passwords> --data-binary "@test_mapping.json"

curl -H "Content-Type: application/x-ndjson" -X PUT "https://localhost:9200/test/_bulk" -ku admin:Kwonjiwon411! --data-binary "@test_data.json"

In [ ]:
index_name = 'test2'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 4
    }
  }
}

response = client.indices.create(index_name, body=index_body)

print(response)

In [33]:
import json

json_file = open('test_data.json', encoding='utf-8')
test_data = json.load(json_file)

tests = ''
id = 1
for line in test_data:
    response = client.index(
        index = 'test2',
        body = line,
        id = str(id),
        refresh = True
    )
    id += 1
    tests = tests + "\n" + str(response)



In [ ]:
client.bulk(tests)

In [ ]:
# keyword search

# korean may not work? 


q = 'news'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print(response)

In [ ]:
# model id CM6SGZIBXZwX3jIm7TxM